## 1.2 读取ROOT的tree数据，进行逐事件分析
### 目的：学习对ROOT文件进行逐事件分析


``` cpp
//将下列代码保存到readTree.cc
//在ROOT环境下 .L readTree.cc

TH1D *hTOF;//如果将hTOF定义写在函数体内 hTOF->Draw()将不会显示
void readTree()
{
// 1.打开文件，得到TTree指针
  TFile *ipf=new TFile("tree.root");//打开ROOT文件
  TTree *tree=(TTree*)ipf->Get("tree");//得到名字为“tree”的TTree指针

//2. 声明tree的Branch变量

  Double_t x;
  Double_t e;
  int ng;
  Double_t tof, ctof;
  Double_t tu, td;
  Double_t qu, qd;
  
//3. 将变量指向对应Branch的地址
  tree->SetBranchAddress("ctof",&ctof);
  tree->SetBranchAddress("ng",&ng);
  tree->SetBranchAddress("tu",&tu);   
  tree->SetBranchAddress("td",&td);
  tree->SetBranchAddress("qu",&qu);   
  tree->SetBranchAddress("qd",&qd);

  //Histogram
  hTOF=new TH1D("hTOF","Time of flight", 1000,0,100);
  
  //将新数据写入新的ROOT文件 -对应的代码 ////标出
  //// //calibration parameters
  //// Double_t a,b;
  //// ... ... ...
  //// //new tree parameters
  //// Double_t tx,qx,ce;
  //// ... ... ...    
  //// TFile *opf=new TFile("tree2.root","recreate");
  //// TTree *opt=new TTree("tree","tree");
  //// opt->SetBrach("tx",&tx,"tx/D");
  //// ... ... ...
    
    
  //4. 逐事件读取tree的branch数据
  Long64_t nentries=tree->GetEntries();//得到事件总数
  for(Long64_t jentry=0; jentry<nentries; jentry++) {//对每个事件进行遍历
    tree->GetEntry(jentry);//将第jentry个事件数据填入对应变量，每次变量值会变成当前事件对应的数据。
    hTOF->Fill(ctof);
    //// // calculate new parameters
    //// tx=tu-td;
    //// ... ... ...
      
    //// opt->Fill();//fill new parameter to TTree* opt
      
    if(jentry%100000==0) cout<<"process "<<jentry<<" of "<<nentries<<endl;
  }

  hTOF->Draw();
  ipt->Close();
  //// opt->Write();
  //// opf->Close();
}

 ```
 ### 练习：
  - 理解上述1.-4.步骤的逻辑关系。自行练习将上一步生成ROOT文件打开逐事件读出数据。
  - 理解如何在模拟中加入探测器分辨率信息。

In [1]:
%jsroot on
TH1D *tx=new TH1D("tx","tx",140,-50,20);  
TCanvas *c1=new TCanvas("c1","c1");

Double_t tu, td;
Double_t ntof,x;
int ng;
TFile *ipf=new TFile("tree.root");//打开ROOT文件
TTree *tree=(TTree*)ipf->Get("tree");//得到tree的指针

tree->SetBranchAddress("tu",&tu);   
tree->SetBranchAddress("td",&td);
tree->SetBranchAddress("nTOF",&ntof);
tree->SetBranchAddress("x",&x);
tree->SetBranchAddress("ng",&ng);

Error in <TTree::SetBranchAddress>: unknown branch -> nTOF


In [2]:
Long64_t nentries=tree->GetEntries();//得到事件总数
for(Long64_t jentry=0; jentry<nentries; jentry++) {//对每个事件进行遍历
    tree->GetEntry(jentry);
    tx->Fill(tu-td);  // if(ng==1) tx->Fill(tu-td), 只写入满足给定条件的事件      
  }
tx->Draw();
c1->Draw();

In [3]:
//确定均匀分布边界的方法-微分法
TH1D *dtx=new TH1D("dtx","dtx",140,-50,20);
for(int i=1;i<tx->GetNbinsX();i++) {
    Double_t df=tx->GetBinContent(i+1)-tx->GetBinContent(i);
    dtx->Fill(tx->GetBinCenter(i),df);
}
dtx->Sumw2(0);
dtx->Draw();
c1->Draw();

#### 探测器时间差-位置的刻度方法
$t_x=t_u-t_d$, $t_{xl}=-41.5, t_{xr}=11.5$ => $t_{xoff}=(t_{xl}+t_{xr})/2=-15.0$.

$x=(t_{xr}-t_{xl})/2L*(t_x-t_{xoff})=3.7735849*(t_x+15.)$

In [4]:
TH1D *hx=new TH1D("hx","hx",500,-120,120);
TH1D *htof=new TH1D("htof","htof",500,0,20);
TH1D *hntof=new TH1D("hntof","hntof",500,0,20);
TH2D *htofx=new TH2D("htofx","htofx",500,-120,120,500,0,20);
for(Long64_t jentry=0; jentry<nentries; jentry++) {//对每个事件进行遍历
    tree->GetEntry(jentry);
    Double_t xx=3.7735849*(tu-td+15.);
    hx->Fill(xx);
    Double_t xxL=TMath::Sqrt(502.5*502.5+xx*xx)/100.;
    Double_t tofa=(tu+td)/2./xxL;//normalized to 100cm
    htof->Fill(tofa);
    hntof->Fill(ntof/500*100.);
    htofx->Fill(xx,tofa);
  }
hx->Draw();
c1->Draw();


In [5]:
hntof->SetLineColor(kRed);
htof->Draw();
c1->Draw();

在飞行距离为100cm时，gamma的峰位在8.16ns，说明tof的offset在8.16-3=5.16ns, 以靶为时间起点的飞行时间为tof-5.16ns

In [6]:
htofx->Draw("colz");
c1->Draw();